In [ ]:
!pip install datasets

## Part 1

In [ ]:
# Question 1(a): What is the size of the vocabulary formed from your training data?

from datasets import load_dataset
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Load the dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset['train']

# Initialize an empty set to store unique words
vocabulary = set()

# Tokenize each review in the training dataset and update the vocabulary set
for text in train_dataset['text']:
    tokens = word_tokenize(text.lower())
    vocabulary.update(tokens)

# Print the size of the vocabulary
print("The size of the vocabulary is:", len(vocabulary))

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# Question 1(b): How many OOV words exist in your training data?

import numpy as np

# Load the GloVe embeddings (make sure to download 'glove.6B.100d.txt' and place it in the working directory)
glove_vocab = set()
with open('glove.6B.100d.txt', 'r', encoding='utf8') as f:
    for line in f:
        word = line.split()[0]
        glove_vocab.add(word)

# Identify OOV words
oov_words = vocabulary - glove_vocab

# Print the number of OOV words
print("Number of OOV words in the training data:", len(oov_words))

NameError: name 'vocabulary' is not defined

In [ ]:
# Question 1(c): Mitigating OOV Words by Initializing Random Embeddings

embedding_dim = 100  # Dimensionality of GloVe embeddings
import numpy as np

# Create mappings between words and indices
word2idx = {}
idx2word = {}
for idx, word in enumerate(vocabulary):
    word2idx[word] = idx
    idx2word[idx] = word

# Initialize the embedding matrix with random values
embedding_matrix = np.random.uniform(-0.05, 0.05, (len(vocabulary), embedding_dim))

# Load GloVe embeddings into the embedding matrix where possible
with open('glove.6B.100d.txt', 'r', encoding='utf8') as f:
    for line in f:
        values = line.split()
        glove_word = values[0]
        if glove_word in word2idx:
            vector = np.asarray(values[1:], dtype='float32')
            idx = word2idx[glove_word]
            embedding_matrix[idx] = vector

# Now, embedding_matrix contains GloVe embeddings for known words and random values for OOV words
print("Embedding matrix shape:", embedding_matrix.shape)

Embedding matrix shape: (18029, 100)


## Part 2

In [ ]:
# Import necessary libraries
from datasets import load_dataset
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn

# Load the dataset
dataset = load_dataset("rotten_tomatoes")
train_raw = dataset['train']
validation_raw = dataset['validation']
test_raw = dataset['test']

# Build the vocabulary from training data
vocabulary = set()
for text in train_raw['text']:
    tokens = word_tokenize(text.lower())
    vocabulary.update(tokens)

# Add special tokens
vocabulary.add('<unk>')
vocabulary.add('<pad>')

# Create mappings between words and indices
word2idx = {word: idx for idx, word in enumerate(vocabulary)}
idx2word = {idx: word for word, idx in word2idx.items()}

# Initialize the embedding matrix
embedding_dim = 100  # Dimensionality of GloVe embeddings
vocab_size = len(word2idx)
embedding_matrix = np.random.uniform(-0.05, 0.05, (vocab_size, embedding_dim))

# Load GloVe embeddings (ensure 'glove.6B.100d.txt' is in your working directory)
glove_path = 'glove.6B.100d.txt'  # Update the path if necessary
with open(glove_path, 'r', encoding='utf8') as f:
    for line in f:
        values = line.strip().split()
        if len(values) == embedding_dim + 1:
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            if word in word2idx:
                idx = word2idx[word]
                embedding_matrix[idx] = vector

# Set the embedding for '<pad>' token to zeros
pad_idx = word2idx['<pad>']
embedding_matrix[pad_idx] = np.zeros(embedding_dim)

# Define a function to convert sentences to indices
def sentence_to_indices(sentence, word2idx):
    tokens = word_tokenize(sentence.lower())
    indices = []
    for token in tokens:
        if token in word2idx:
            indices.append(word2idx[token])
        else:
            indices.append(word2idx['<unk>'])  # Map unknown words to '<unk>'
    return indices

# Define the custom Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, word2idx):
        self.texts = texts
        self.labels = labels
        self.word2idx = word2idx

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        indices = sentence_to_indices(text, self.word2idx)
        return torch.tensor(indices, dtype=torch.long), torch.tensor(label, dtype=torch.long)

# Create datasets for training, validation, and testing
train_dataset = SentimentDataset(train_raw['text'], train_raw['label'], word2idx)
val_dataset = SentimentDataset(validation_raw['text'], validation_raw['label'], word2idx)
test_dataset = SentimentDataset(test_raw['text'], test_raw['label'], word2idx)

# Define the collate_fn function for padding within batches
def collate_fn(batch):
    sequences = [item[0] for item in batch]
    labels = torch.tensor([item[1] for item in batch], dtype=torch.float)
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=word2idx['<pad>'])
    return sequences_padded, labels

# Create DataLoaders for training, validation, and testing
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Define the RNN model using pre-trained embeddings
class SentimentRNN(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentRNN, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float32), freeze=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (h_n, c_n) = self.lstm(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()


# Instantiate the model
model = SentimentRNN(embedding_matrix)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, factor=0.5, verbose=True)

# Training loop with validation and early stopping
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  # Gradient clipping
        optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break



# Load the best model and evaluate on the test set
model.load_state_dict(torch.load('best_model.pt'))
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
test_accuracy = correct / total
print(f'Test Accuracy: {test_accuracy:.4f}')

[nltk_data] Downloading package punkt to C:\Users\Samuel
[nltk_data]     Ng\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\Samuel Ng\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/30], Loss: 0.6354, Validation Accuracy: 0.6979
Epoch [2/30], Loss: 0.5430, Validation Accuracy: 0.7026
Epoch [3/30], Loss: 0.5229, Validation Accuracy: 0.7364
Epoch [4/30], Loss: 0.5023, Validation Accuracy: 0.7598
Epoch [5/30], Loss: 0.4745, Validation Accuracy: 0.7364
Epoch [6/30], Loss: 0.4521, Validation Accuracy: 0.7580
Epoch [7/30], Loss: 0.4301, Validation Accuracy: 0.7392
Epoch [8/30], Loss: 0.3914, Validation Accuracy: 0.7655
Epoch [9/30], Loss: 0.3673, Validation Accuracy: 0.7711
Epoch [10/30], Loss: 0.3432, Validation Accuracy: 0.7767
Epoch [11/30], Loss: 0.3233, Validation Accuracy: 0.7692
Epoch [12/30], Loss: 0.3010, Validation Accuracy: 0.7767
Epoch [13/30], Loss: 0.2818, Validation Accuracy: 0.7795
Epoch [14/30], Loss: 0.2546, Validation Accuracy: 0.7805
Epoch [15/30], Loss: 0.2361, Validation Accuracy: 0.7786
Epoch [16/30], Loss: 0.2158, Validation Accuracy: 0.7730
Epoch [17/30], Loss: 0.1837, Validation Accuracy: 0.7467
Epoch [18/30], Loss: 0.1455, Validation 

In [ ]:
#3.1

In [ ]:
class SentimentRNN_UpdateEmbeddings(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentRNN_UpdateEmbeddings, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True  # Update the embeddings during training
        self.lstm = nn.LSTM(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (h_n, c_n) = self.lstm(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()

In [ ]:
# Instantiate the model
model = SentimentRNN_UpdateEmbeddings(embedding_matrix)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, factor=0.5, verbose=True)

# Training loop with validation and early stopping
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  # Gradient clipping
        optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model_update_embeddings.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break

# Load the best model and evaluate on the test set
model.load_state_dict(torch.load('best_model_update_embeddings.pt'))
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
test_accuracy = correct / total
print(f'Test Accuracy: {test_accuracy:.4f}')

Epoch [1/30], Loss: 0.6158, Validation Accuracy: 0.7223
Epoch [2/30], Loss: 0.4818, Validation Accuracy: 0.7608
Epoch [3/30], Loss: 0.3559, Validation Accuracy: 0.7552
Epoch [4/30], Loss: 0.2421, Validation Accuracy: 0.7645
Epoch [5/30], Loss: 0.1518, Validation Accuracy: 0.7608
Epoch [6/30], Loss: 0.0922, Validation Accuracy: 0.7655
Epoch [7/30], Loss: 0.0494, Validation Accuracy: 0.7664
Epoch [8/30], Loss: 0.0263, Validation Accuracy: 0.7542
Epoch [9/30], Loss: 0.0151, Validation Accuracy: 0.7570
Epoch [10/30], Loss: 0.0092, Validation Accuracy: 0.7448
Epoch [11/30], Loss: 0.0038, Validation Accuracy: 0.7477
Epoch [12/30], Loss: 0.0013, Validation Accuracy: 0.7448
Early stopping!
Test Accuracy: 0.7617


In [ ]:
#3.2

In [ ]:
# Load the GloVe embeddings
glove_vocab = set()
with open('glove.6B.100d.txt', 'r', encoding='utf8') as f:
    for line in f:
        word = line.split()[0]
        glove_vocab.add(word)

# Initialize the embedding matrix with random values
embedding_dim = 100
vocab_size = len(vocabulary)
embedding_matrix = np.random.uniform(-0.05, 0.05, (vocab_size, embedding_dim))

# Create a dictionary to store the GloVe embeddings
glove_embeddings = {}
glove_file = 'glove.6B.100d.txt'
with open(glove_file, 'r', encoding='utf8') as f:
    for line in f:
        values = line.split()
        glove_word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        # Normalize the loaded vectors to have unit norm
        vector /= np.linalg.norm(vector)
        glove_embeddings[glove_word] = vector

# Load GloVe embeddings into the embedding matrix where possible
for idx, word in enumerate(vocabulary):
    if word in glove_embeddings:
        embedding_matrix[idx] = glove_embeddings[word]

In [ ]:
class SentimentRNN_OOV(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentRNN_OOV, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.lstm = nn.LSTM(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (h_n, c_n) = self.lstm(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()

In [ ]:

# Instantiate the model
model = SentimentRNN_OOV(embedding_matrix)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2, factor=0.5, verbose=True)

# Training loop with validation and early stopping
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  # Gradient clipping
        optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model_oov.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break


Epoch [1/30], Loss: 0.6444, Validation Accuracy: 0.7514
Epoch [2/30], Loss: 0.4196, Validation Accuracy: 0.7552
Epoch [3/30], Loss: 0.2322, Validation Accuracy: 0.7495
Epoch [4/30], Loss: 0.1236, Validation Accuracy: 0.7542
Epoch [5/30], Loss: 0.0663, Validation Accuracy: 0.7392
Epoch [6/30], Loss: 0.0291, Validation Accuracy: 0.7458
Epoch [7/30], Loss: 0.0119, Validation Accuracy: 0.7477
Early stopping!


In [ ]:
# Load the best model and evaluate on the test set
model.load_state_dict(torch.load('best_model_oov.pt'))
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
test_accuracy = correct / total
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.7720


In [ ]:
# biLSTM Model
class SentimentBiLSTM(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentBiLSTM, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.lstm = nn.LSTM(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (h_n, c_n) = self.lstm(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()

In [ ]:
# biLSTM Model
class SentimentBiLSTM(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentBiLSTM, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.lstm = nn.LSTM(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, (h_n, c_n) = self.lstm(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()


# biGRU Model
class SentimentBiGRU(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentBiGRU, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.gru = nn.GRU(embedding_dim, hidden_size=256, batch_first=True, bidirectional=True, num_layers=2, dropout=0.5)
        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        gru_out, h_n = self.gru(embeds)

        # Concatenate the final hidden states from both directions
        out = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)

        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()




In [ ]:
# Instantiate the models
bilstm_model = SentimentBiLSTM(embedding_matrix)
bigru_model = SentimentBiGRU(embedding_matrix)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bilstm_model.to(device)
bigru_model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
bilstm_optimizer = torch.optim.Adam(bilstm_model.parameters(), lr=0.001, weight_decay=1e-5)
bigru_optimizer = torch.optim.Adam(bigru_model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(bilstm_optimizer, 'max', patience=2, factor=0.5, verbose=True)

In [ ]:
# Training loop with validation and early stopping for biLSTM
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    bilstm_model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        bilstm_optimizer.zero_grad()
        outputs = bilstm_model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bilstm_model.parameters(), max_norm=1)  # Gradient clipping
        bilstm_optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    bilstm_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = bilstm_model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(bilstm_model.state_dict(), 'best_bilstm_model.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break

# Load the best model and evaluate on the test set for biLSTM
bilstm_model.load_state_dict(torch.load('best_bilstm_model.pt'))
bilstm_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = bilstm_model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
bilstm_test_accuracy = correct / total
print(f'biLSTM Test Accuracy: {bilstm_test_accuracy:.4f}')

Epoch [1/30], Loss: 0.6494, Validation Accuracy: 0.7345
Epoch [2/30], Loss: 0.4284, Validation Accuracy: 0.7523
Epoch [3/30], Loss: 0.2194, Validation Accuracy: 0.7411
Epoch [4/30], Loss: 0.1183, Validation Accuracy: 0.7570
Epoch [5/30], Loss: 0.0712, Validation Accuracy: 0.7683
Epoch [6/30], Loss: 0.0434, Validation Accuracy: 0.7439
Epoch [7/30], Loss: 0.0275, Validation Accuracy: 0.7580
Epoch [8/30], Loss: 0.0184, Validation Accuracy: 0.7561
Epoch [9/30], Loss: 0.0102, Validation Accuracy: 0.7561
Epoch [10/30], Loss: 0.0059, Validation Accuracy: 0.7552
Early stopping!
biLSTM Test Accuracy: 0.7795


In [ ]:
# Training loop with validation and early stopping for biGRU
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    bigru_model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        bigru_optimizer.zero_grad()
        outputs = bigru_model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bigru_model.parameters(), max_norm=1)  # Gradient clipping
        bigru_optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    bigru_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = bigru_model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(bigru_model.state_dict(), 'best_bigru_model.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break

# Load the best model and evaluate on the test set for biGRU
bigru_model.load_state_dict(torch.load('best_bigru_model.pt'))
bigru_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = bigru_model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
bigru_test_accuracy = correct / total
print(f'biGRU Test Accuracy: {bigru_test_accuracy:.4f}')

Epoch [1/30], Loss: 0.6209, Validation Accuracy: 0.7176
Epoch [2/30], Loss: 0.3693, Validation Accuracy: 0.7523
Epoch [3/30], Loss: 0.1926, Validation Accuracy: 0.7580
Epoch [4/30], Loss: 0.0981, Validation Accuracy: 0.7486
Epoch [5/30], Loss: 0.0543, Validation Accuracy: 0.7439
Epoch [6/30], Loss: 0.0264, Validation Accuracy: 0.7289
Epoch [7/30], Loss: 0.0156, Validation Accuracy: 0.7355
Epoch [8/30], Loss: 0.0113, Validation Accuracy: 0.7383
Early stopping!
biGRU Test Accuracy: 0.7795


In [ ]:
# CNN Model
class SentimentCNN(nn.Module):
    def __init__(self, embedding_matrix):
        super(SentimentCNN, self).__init__()
        vocab_size, embedding_dim = embedding_matrix.shape
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.conv1 = nn.Conv1d(embedding_dim, 128, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x).permute(0, 2, 1)
        conv_out = self.conv1(embeds)
        pool_out = torch.max(conv_out, dim=-1)[0]
        out = self.dropout(pool_out)
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        out = torch.sigmoid(self.fc2(out))
        return out.squeeze()

In [ ]:
# Instantiate the model
cnn_model = SentimentCNN(embedding_matrix)

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn_model.to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
cnn_optimizer = torch.optim.Adam(cnn_model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(cnn_optimizer, 'max', patience=2, factor=0.5, verbose=True)

# Training loop with validation and early stopping
num_epochs = 30
patience = 5  # Early stopping patience
best_val_accuracy = 0
epochs_no_improve = 0

for epoch in range(num_epochs):
    cnn_model.train()
    running_loss = 0.0
    for sequences, labels in train_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        cnn_optimizer.zero_grad()
        outputs = cnn_model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(cnn_model.parameters(), max_norm=1)  # Gradient clipping
        cnn_optimizer.step()
        running_loss += loss.item() * sequences.size(0)
    epoch_loss = running_loss / len(train_dataset)

    # Validation
    cnn_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            outputs = cnn_model(sequences)
            predicted = (outputs >= 0.5).long()
            correct += (predicted == labels.long()).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Learning rate scheduling
    scheduler.step(val_accuracy)

    # Check for improvement
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        epochs_no_improve = 0
        # Save the best model
        torch.save(cnn_model.state_dict(), 'best_cnn_model.pt')
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print('Early stopping!')
            break

# Load the best model and evaluate on the test set
cnn_model.load_state_dict(torch.load('best_cnn_model.pt'))
cnn_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device)
        labels = labels.to(device)
        outputs = cnn_model(sequences)
        predicted = (outputs >= 0.5).long()
        correct += (predicted == labels.long()).sum().item()
        total += labels.size(0)
cnn_test_accuracy = correct / total
print(f'CNN Test Accuracy: {cnn_test_accuracy:.4f}')

C:\Users\Samuel Ng\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch [1/30], Loss: 0.6628, Validation Accuracy: 0.7270
Epoch [2/30], Loss: 0.4245, Validation Accuracy: 0.7645
Epoch [3/30], Loss: 0.1890, Validation Accuracy: 0.7711
Epoch [4/30], Loss: 0.0689, Validation Accuracy: 0.7608
Epoch [5/30], Loss: 0.0259, Validation Accuracy: 0.7467
Epoch [6/30], Loss: 0.0147, Validation Accuracy: 0.7448
Epoch [7/30], Loss: 0.0059, Validation Accuracy: 0.7477


C:\Users\Samuel Ng\anaconda3\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [8/30], Loss: 0.0047, Validation Accuracy: 0.7523
Early stopping!
CNN Test Accuracy: 0.7805
